<a href="https://colab.research.google.com/github/VRamBalla/BME547_Classwork/blob/main/Project2_Team5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NOTE: This notebook assumes that the folder "team_5" is placed as a shortcut in your drive. 

#Install and Import Modules

In [ ]:
!pip install exrex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import os
import sys
dataset_path = '/content/gdrive/My Drive/team_5/project_2/'
sys.path.append(dataset_path)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pickle
import exrex
import re

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, Dropout, MaxPooling2D, Flatten, Dense
from tensorflow.keras import layers
from keras import models
from tqdm import tqdm
import math

#Fine-Tuning ProtGPT2 on Cas9 Sequences
This section of code allows us to pass Cas9 sequence data into ProtGPT2, so that it may change its weights in a directed fashion.

In [ ]:
data_8nt = pd.read_csv(dataset_path + 'Final_Cleaned_Dataset.csv') #This reads in the Cas9 dataset provided by Dr. Chatterjee

protein_sequences = data_8nt['AA Sequence'].values.tolist() #Puts all of the amino acid sequences into a list
seq_len = list(data_8nt['Sequence Length']) #Creates a list of the lenght of every amino acid sequence
max_seq_len = np.max(seq_len) #The maximum amino acid sequence length in the dataset

In [ ]:
data_8nt['AA Sequence'] = data_8nt['AA Sequence'].apply(lambda x: '<|endoftext|>'+x+'<|endoftext|>') #We add the <|endoftext|> string to each end of the protein sequences so that they may be used in the fine-tuning of ProtGPT2

train_seq, validate_seq, _, _ = train_test_split(data_8nt['AA Sequence'], np.zeros((data_8nt.shape[0], 1)), train_size=0.8) #Generate training, and validation data for the fine-tuning of ProtGPT2

np.savetxt('/content/gdrive/My Drive/team_5/project_2/ProtGPT2/ProtGPT2_train.txt', train_seq.values, fmt='%s') #Saves the training and validation data as txt files
np.savetxt('/content/gdrive/My Drive/team_5/project_2/ProtGPT2/ProtGPT2_val.txt', validate_seq.values, fmt='%s')

In [ ]:
!pip install -r '/content/gdrive/My Drive/team_5/project_2/ProtGPT2/requirements.txt' #Installs the necessary requirements for ProtGPT2


The line below is the training of the ProtGPT2 model. A batch size of 2 was utilized, with a learning rate of 1e-05 using the Adam optimizer

In [ ]:
#!python '/content/gdrive/My Drive/team_5/project_2/ProtGPT2/run_clm.py' --model_name_or_path nferruz/ProtGPT2 --per_device_train_batch_size 2 --train_file '/content/gdrive/My Drive/team_5/project_2/ProtGPT2/ProtGPT2_train.txt' --validation_file '/content/gdrive/My Drive/team_5/project_2/ProtGPT2/ProtGPT2_val.txt' --tokenizer_name nferruz/ProtGPT2 --do_train --do_eval --output_dir '/content/gdrive/My Drive/team_5/project_2/ProtGPT2/ProtGPT2_output'  --learning_rate 1e-05 --overwrite_output_dir

#Generation of New Sequences
The following lines of code generate novel sequences from our fine-tuned ProtGPT2 model. We generated 100 sequences.

This code was only run once since we only needed to fine-tune and generate sequences from ProtGPT2 one time

In [ ]:
from transformers import pipeline 
protgpt2 = pipeline('text-generation', model= dataset_path + 'ProtGPT2/ProtGPT2_output/') #here, we are initializing the fine-tuned ProtGPT2 model that was created from the previous training step.


We generated 100 amino acid sequences from the fine-tuned model. We have put the restriciton that the amino acid sequence must start with M, as this is the start codon amino acid.

The final line prints all of the generated sequences; note that every 60 amino acids, there is a \n entry. This is consistent with the FASTA file format, and is used by ProtGPT2 later in the perplexity calculations

In [ ]:
sequences = protgpt2("M", max_length=max_seq_len, do_sample=True, top_k=950, repetition_penalty=1.2, num_return_sequences=100, eos_token_id=0)
for seq in sequences:
        print(seq) 

In [ ]:
gen_seq=[] #Initialize a list in which the generated sequences will be stored
for i in range(0,len(sequences)):
  gen_seq.append(sequences[i]['generated_text']) #Append each generated protein sequence to the list

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import math

tokenizer = GPT2Tokenizer.from_pretrained(dataset_path + 'ProtGPT2/ProtGPT2_output/') #Load in the tokenizer from the fine-tuned ProtGPT2 Model
model = GPT2LMHeadModel.from_pretrained(dataset_path + 'ProtGPT2/ProtGPT2_output/') #Load in the stored fine-tuned ProtGPT2 model

#Function to calculate perplexity
def calculatePerplexity(sequence, model, tokenizer):
    input_ids = torch.tensor(tokenizer.encode(sequence)).unsqueeze(0) 
    input_ids = input_ids
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    loss, logits = outputs[:2]
    return math.exp(loss)


ppl = [] #Initialize list for storage of perplexity values
for i in range(0,len(gen_seq)):
  sequence=gen_seq[i]
  sequence = '<|endoftext|>' + sequence + '<|endoftext|>' #Add these delimiter strings at the beginning and end of every amino acid sequence
  ppl.append(calculatePerplexity(sequence, model, tokenizer)) #Append the perplexity value of each generated sequences to the perplexity list


In [ ]:
clean_sequences = [] #This list will store the "clean" version of the generated sequence; without the \n separators in the previous FASTA format

for i in range(0,len(gen_seq)):
  s = gen_seq[i]
  n = s.split('\n') #Split the generated sequence by all instances of "\n"
  s_new = ''.join(n) #Join the split parts of the sequence back together. The result is a string that only consists of amino acids
  clean_sequences.append(s_new) #Append the clean sequence to the clean_sequences list

generated_sequences = pd.DataFrame( #Initialize a dataframe in which the generated amino acid sequences and perplexity values will be stored
    {'AA Sequence':clean_sequences,
     'Perplexity':ppl})

In [ ]:
sorted_generated_sequences = generated_sequences.sort_values(by = 'Perplexity') #Sort the amino acid sequences by their perplexity value; the lower the perplexity the value, the more confident we are in the output of the fine-tuned ProtGPT2 Model


The following line of code is commented out because it writes the sorted_generated_sequences DataFrame to a csv file; there is no need to repeat this everytime the notebook is run.

In [ ]:
# sorted_generated_sequences.to_csv('/content/gdrive/My Drive/team_5/project_2/GeneratedSequences.csv')

#PAM Prediction of Generated Cas9 Sequences Using Convolutional Neural Networks
8 CNNs using the same model architecture were run: each time the VHSE-embedded protein inputs remained the same, but the 2D PAM labels were swapped out for each run of the model.

###Training the CNNs

#### PAM Encoding for CNN
This section contains the code by which PAM sequences were encoded in n x 4 matrices for use as a multi-label output in our 8 CNN models (1 model for each nucleotide position).

The below dictionary assigns each of the base 4 nucleotides to a number; this is the column index number that will later be used when constructing the PAM encoded arrays.

In [ ]:
nt_dict = {'A':0,'C':1,'T':2,'G':3} #Create a matrix where each column corresponds to one of the 4 nucleotides in DNA

In [ ]:
#Create list for each nucleotide in PAM sequence
nt1 = data_8nt['nt1'].tolist()
nt2 = data_8nt['nt2'].tolist()
nt3 = data_8nt['nt3'].tolist()
nt4 = data_8nt['nt4'].tolist()
nt5 = data_8nt['nt5'].tolist()
nt6 = data_8nt['nt6'].tolist()
nt7 = data_8nt['nt7'].tolist()
nt8 = data_8nt['nt8'].tolist()

The dictionary below assigns all of the IUPAC ambiguity codes to their meanings in terms of the base 4 nucleotides. Thus, coupling this with the expand_PAM function defined below outputs a list with all of the possible DNA sequences only in terms of A,C,T,G even if the input sequence contains an ambiguity code.

In [ ]:
IUPAC_r = { #Create a dictionary for all of the ambiguous DNA codes, mapping them to their equivalent possibilites in terms of the 4 fundamnetal nucleotides
    'N': '(A|C|T|G)',
    'R': '(G|A)',
    'Y': '(T|C)',
    'M': '(A|C)',
    'K': '(G|T)',
    'S': '(G|C)',
    'W': '(A|T)',
    'H': '(A|C|T)',
    'B': '(G|C|T)',
    'V': '(A|C|G)',
    'D': '(A|G|T)'
}

ambiguity_codes = list(IUPAC_r.keys()) #Create a list of the ambiguity codes

#This function returns all of the possible PAMs if that PAM contains an ambiguity code
#For example, NATG would return AATG, CATG, CATG, and TATG
def expand_PAM(seq):
  for s in seq:
    if s in IUPAC_r:
      seq = seq.replace(s, IUPAC_r[s])
  seq = list(exrex.generate(seq))
  return seq

The below function constructs the 2D PAM encoding array for each nucleotide position in the PAM sequence

In [ ]:
def nt_encode(ntlist):
  a = np.zeros((len(ntlist),4)) #Initialize an array that has number of rows equal to the number of samples passed into the function
  for i in range(0,len(ntlist)):
    if type(ntlist[i]) == str:
      if ntlist[i] in ambiguity_codes:
        e = expand_PAM(ntlist[i]) #Utilize the expand_PAM function if the nucleotide is an ambigiuity code
        for x in e:
         a[i][nt_dict[x]] = 1 #Place the value 1 in all of the relavent columns corresponding to the fundamental nucleotide possibilites
      else:
        a[i][nt_dict[ntlist[i]]] = 1 #If the nucleotide is not an ambiguity code, just place a 1 in the one column for that nucleotide

  return a

In [ ]:
#Generate 2D PAM encoding arrays for each nucleotide position in the PAM sequence
nt1_encoded = nt_encode(nt1)
nt2_encoded = nt_encode(nt2)
nt3_encoded = nt_encode(nt3)
nt4_encoded = nt_encode(nt4)
nt5_encoded = nt_encode(nt5)
nt6_encoded = nt_encode(nt6)
nt7_encoded = nt_encode(nt7)
nt8_encoded = nt_encode(nt8)

### VHSE Embedding of Protein Sequences from the original dataset
This function takes in an amino acid sequence, splits the string into its component characters, and assembles a VHSE encoded matrix. Each row of the matrix represents one amino acid in the protein sequence, and there are 8 columns for each of the 8 VHSE values. The function also takes as an input the maximum length of a protein sequence so that adequate zero-padding can be applied to make sure all inputs are of the same size.

In [ ]:
#Using pandas to read in the csv file containing the VHSE encoding values for each amino acid
VHSE_val = pd.read_csv(dataset_path + 'VHSE8.csv')

def VHSE_encode(seq, max_len):
    s = [*seq] #Split the amino acid sequence for each character in the string
    m = []
    for i in range(0,len(s)):
        for l in range(0,np.shape(VHSE_val)[0]):
            if s[i] == VHSE_val['Single Code'][l]:
                m.append(VHSE_val.iloc[l][2:10]) #This adds all 8 VHSE values for a given amino acid
    
    if len(m) != max_len:
       for x in range(0,max_len-len(m)):
         m.append([0,0,0,0,0,0,0,0]) #This applies zero-padding so that the shape of all encoded protein sequences are the same. The amount of zero padding is based on the maximum protein sequence length

    a = np.array(m).astype('float32')

    return a

The VHSE takes approximately 30 minutes to run due to the large volume of data. Thus, instead of having to re-embed the protein sequences every time we start a new runtime, the VHSE encoded protein sequences were saved as a csv file that could then be loaded and reshaped into the 3D array

In [ ]:
#VHSE_encoded = [VHSE_encode(seq,max_seq_len) for seq in protseqs]
#VHSE_encoded = np.array(VHSE_encoded)

The two functions defined below allow for the export and import of the VHSE-embedded csv file.

In [ ]:
def export_matrix(a,file): #This function exports the 3D-dimensional VHSE array by first flattening it and writing it to a txt file
    a_reshaped = a.reshape(a.shape[0],-1)
    np.savetxt(file,a_reshaped)

def import_matrix(file,twoshape): #This functions loads in the txt file and reshapes it back into the 3D numpy array
  loaded_a = np.loadtxt(file)
  load_original_a = loaded_a.reshape(loaded_a.shape[0],loaded_a.shape[1] // twoshape, twoshape)
  return load_original_a

This code block has been commented because the csv export has already happened

In [ ]:
 #export_matrix(VHSE_encoded,'/content/drive/My Drive/team_5/project_1/Data/FinalData_VHSE_Encoded.csv')

The final embedding result is a 3D array, where each sample is encoded as a 1679 x 8 matrix

In [ ]:
VHSE_encoded = import_matrix(dataset_path + 'FinalData_VHSE_Encoded.csv',8)

####Reshaping the VHSE Protein Embedding
The VHSE encoded array is reshaped so that it can be a tensor, which is the required input for the CNN models.

In [ ]:
VHSE_encoded_tensor = np.expand_dims(VHSE_encoded,-1)

####Splitting the Data for CNN
The data for each nucleotide position was split as 72/8/20 for train/val/test. The VHSE-encoded protein sequences remain constant across all models, only the PAM nucleotide position labels are changing.

In [ ]:
train_seq, test_seq, train_nt1, test_nt1 = train_test_split(VHSE_encoded_tensor,nt1_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt1, val_nt1 = train_test_split(train_seq, train_nt1, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt2, test_nt2 = train_test_split(VHSE_encoded_tensor,nt2_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt2, val_nt2 = train_test_split(train_seq, train_nt2, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt3, test_nt3 = train_test_split(VHSE_encoded_tensor,nt3_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt3, val_nt3 = train_test_split(train_seq, train_nt3, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt4, test_nt4 = train_test_split(VHSE_encoded_tensor,nt4_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt4, val_nt4 = train_test_split(train_seq, train_nt4, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt5, test_nt5 = train_test_split(VHSE_encoded_tensor,nt5_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt5, val_nt5 = train_test_split(train_seq, train_nt5, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt6, test_nt6 = train_test_split(VHSE_encoded_tensor,nt6_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt6, val_nt6 = train_test_split(train_seq, train_nt6, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt7, test_nt7 = train_test_split(VHSE_encoded_tensor,nt7_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt7, val_nt7 = train_test_split(train_seq, train_nt7, test_size=0.1, random_state=42)

train_seq, test_seq, train_nt8, test_nt8 = train_test_split(VHSE_encoded_tensor,nt8_encoded, test_size=0.2, random_state = 2)
train_seq, val_seq, train_nt8, val_nt8 = train_test_split(train_seq, train_nt8, test_size=0.1, random_state=42)

print(np.shape(train_seq))
print(np.shape(train_nt1))

(4508, 1697, 8, 1)
(4508, 4)


###CNN Model Architecture and Training
The CNN model architecutre consists of a convolution layer with 32 kernels of size (3,3), followed by a 2x2 maxpool layer, followed by another convolution layer with 64 kernels of size (3,3), followed by another 2x2 maxpool layer, followed by a flatten layer, dropout of 0.6, and a final dense layer.

In [ ]:
input_shape = (1697,8,1)
num_classes = 4


model = keras.Sequential(
    
    [
        keras.Input(shape = input_shape),
        layers.Conv2D(32, kernel_size = (3,3), padding='same',activation="relu"), #convolutional layer using 32 kernels of size (3,3) with the relu activation unit
        layers.MaxPooling2D(pool_size = (2, 2),padding='same'), #Max pool layer of size (2,2)
        layers.Conv2D(64, kernel_size = (3,3), padding='same',activation="relu"),#convolutional layer using 32 kernels of size (3,3) with the relu activation unit
        layers.MaxPooling2D(pool_size = (2, 2),padding='same'), #Max pool layer of size (2,2)
        layers.Flatten(), #Flattens the output of the final maxpool layer
        layers.Dropout(0.6), #Dropout of 0.6, which randomly drops 60% of nodes
        layers.Dense(num_classes, activation="softmax") #Dense layer in which the final classification predictions are made
    ]

)

model.compile(optimizer='adam', loss='BinaryCrossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 1697, 8, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 849, 4, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 849, 4, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 425, 2, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 54400)             0         
                                                                 
 dropout (Dropout)           (None, 54400)             0

A function was defined so that all 8 CNN models (1 for each nucleotide position) could be run with relative ease. The outputs display the loss and accuracy for the training and validation sets during training, and the test loss and accuracy during the model test.

In [ ]:
def CNN(model,train_input,train_labels,val_input,val_labels,test_input,test_labels,name):
  history = model.fit(x= train_input,
                      y= train_labels,
                      batch_size=60, 
                      callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],
                      validation_data = (val_input,val_labels),
                      verbose = 1,
                      epochs = 50)
  
  score = model.evaluate(x = test_input, 
                             y = test_labels,
                             verbose=1)
  
#   model_json = model.to_json()
#   with open(dataset_path+'VHSE_Encoded CNN Models/'+name+'.json', "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
#     model.save_weights(dataset_path+'VHSE_Encoded CNN Models/'+name+'_weights'+".h5")
#   print("Saved model to disk")
  model.save(dataset_path+'VHSE_Encoded CNN Models/'+name+'/')
  
  print('Test loss:', score[0]) 
  print('Test accuracy:', score[1])

Train the 8 CNN models; 1 for each nucleotide

In [ ]:
CNN(model,train_seq,train_nt1,val_seq,val_nt1,test_seq,test_nt1,'Nucleotide1')

In [ ]:
CNN(model,train_seq,train_nt2,val_seq,val_nt2,test_seq,test_nt2,'Nucleotide2')

In [ ]:
CNN(model,train_seq,train_nt3,val_seq,val_nt3,test_seq,test_nt3,'Nucleotide3')

In [ ]:
CNN(model,train_seq,train_nt4,val_seq,val_nt4,test_seq,test_nt4,'Nucleotide4')

In [ ]:
CNN(model,train_seq,train_nt5,val_seq,val_nt5,test_seq,test_nt5,'Nucleotide5')

In [ ]:
CNN(model,train_seq,train_nt6,val_seq,val_nt6,test_seq,test_nt6,'Nucleotide6')

In [ ]:
CNN(model,train_seq,train_nt7,val_seq,val_nt7,test_seq,test_nt7,'Nucleotide7')

In [ ]:
CNN(model,train_seq,train_nt8,val_seq,val_nt8,test_seq,test_nt8,'Nucleotide8')

###Predicting the PAMs of the ProtGPT2-generated Cas9 Sequences

#####VHSE Embedding of the generated Cas9 Sequences

We decided to only move foward with sequences that had calculated perplexity scores less than 100. Thus, we are left with 18 de novo sequences that will be VHSE encoded and PAM-predicted with the trained CNN models.

In [ ]:
novel_seqs = pd.read_csv(dataset_path + 'GeneratedSequences.csv')
novel_sequences = novel_seqs.query('Perplexity <= 100')['AA Sequence'].tolist() #Only those sequences with perplexity scores are kept

In [ ]:
generated_VHSE_encoded = [VHSE_encode(seq,max_seq_len) for seq in novel_sequences] #VHSE encode the generated sequences with perplexity scores less than 100
generated_VHSE_encoded = np.array(generated_VHSE_encoded)
generated_VHSE_encoded_tensor = np.expand_dims(generated_VHSE_encoded,-1) #The VHSE encoded is expanded to a tensor so that it may be passed into the CNN models

#####Predicting PAMs of the Generated Sequences

In [ ]:
from torch.serialization import DEFAULT_PROTOCOL
# load a pre-trained model to predict on new data
from tensorflow.keras.models import load_model, model_from_json

def predict_model(name):
  prev_model = keras.models.load_model(dataset_path+'VHSE_Encoded CNN Models/'+name) #Loads the saved pre-trained CNN model
  y_predict = prev_model.predict(generated_VHSE_encoded_tensor) #Runs the generated Cas9 sequences through the model for PAM prediction

  df = pd.DataFrame(y_predict) #convert prediction matrix to dataframe
  df.to_csv(dataset_path + 'Generated Sequence PAM Prediction/'+name+'.csv') #save prediction dataframe as csv
  return y_predict


In [ ]:
#Predict the PAM 1 nucleotide at a time for each of the 18 de novo Cas9 sequences, and export the prediction results to csvs
nt1_predict = predict_model('Nucleotide1')
nt2_predict = predict_model('Nucleotide2')
nt3_predict = predict_model('Nucleotide3')
nt4_predict = predict_model('Nucleotide4')
nt5_predict = predict_model('Nucleotide5')
nt6_predict = predict_model('Nucleotide6')
nt7_predict = predict_model('Nucleotide7')
nt8_predict = predict_model('Nucleotide8')

1/1 [==============================] - 0s 65ms/step
